In [441]:
import tensorflow as tf

In [442]:
import pandas as pd

In [443]:
import numpy as np

In [444]:
import itertools

In [445]:
import os

In [446]:
import tempfile

In [447]:
import pickle

In [448]:
from sklearn import datasets

In [449]:
from IPython.display import clear_output

In [450]:
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]

In [451]:
training_set_n = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_train.csv', skipinitialspace=True, skiprows=1, names=COLUMNS)

In [452]:
test_set_n = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_test.csv', skipinitialspace=True, skiprows=1, names=COLUMNS)

In [453]:
prediction_set_n = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_predict.csv', skipinitialspace=True, skiprows=1, names=COLUMNS)

In [454]:
FEATURES =["crim", "zn", "indus", "nox", "rm", "age","dis", "tax", "ptratio"]

In [455]:
LABEL="medv"

In [456]:
feature_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]

In [457]:
print(feature_columns)

[NumericColumn(key='crim', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='zn', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='indus', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='nox', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='rm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='dis', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='tax', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='ptratio', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [458]:
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns, model_dir="train")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdee375d150>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [459]:
def get_input_fn(data_set, num_epochs=None, batch_size=128, shuffle=True, steps=1000):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices(({k: data_set[k].values for k in FEATURES}, data_set[LABEL]))
        if shuffle:
            ds = ds.shuffle(steps)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function     

In [460]:
estimator.train(input_fn=get_input_fn(training_set_n,                                       
                                           num_epochs=None,                                      
                                           batch_size = 128,                                      
                                           shuffle=False),                                      
                                           steps=1000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 20000 into train/model.ckpt.
INFO:tensorflow:loss = 58.184814, step = 20000
INFO:tensorflow:global_step/sec: 204.488
INFO:tensorflow:loss = 58.182087, step = 20100 (0.

In [461]:
ev = estimator.evaluate(    
          input_fn=get_input_fn(test_set_n,                          
          num_epochs=1,                          
          batch_size = 128,                          
          shuffle=False))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-04T00:09:47Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/model.ckpt-21000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2021-01-04-00:09:48
INFO:tensorflow:Saving dict for global step 21000: average_loss = 19.766193, global_step = 21000, label/mean = 22.08, loss = 19.766

In [462]:
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

Loss: 19.766193


In [463]:
y = estimator.predict(    
         input_fn=get_input_fn(prediction_set_n,                          
         num_epochs=1,                          
         batch_size = 128,                          
         shuffle=False))

In [464]:
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/model.ckpt-21000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions: [array([35.078075], dtype=float32), array([20.180553], dtype=float32), array([24.37819], dtype=float32), array([34.91951], dtype=float32), array([14.411057], dtype=float32), array([19.973576], dtype=float32)]


In [465]:
pred_dicts = list(y)
for pred in pred_dicts:
    print(pred["predictions"][0])

In [466]:
tmpdir = tempfile.mkdtemp()
estimator_base_path = os.path.join(tmpdir, 'from_estimator')
feature_spec = tf.feature_column.make_parse_example_spec(feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
estimator_path = estimator.export_saved_model(estimator_base_path, export_input_fn)
imported = tf.saved_model.load(estimator_path)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from train/model.ckpt-21000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /var/folders/6_/8_x7k80s52vcfppnmvb31wxr0000gn/T/tmpj4b5arw0/from_estimator/temp-b'1609718989'/saved_model.pb


In [467]:
def predict(x):
    examples = []
    for index, row in x.iterrows():
        feature = {}
        for col, value in row.iteritems():
            feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        example = tf.train.Example(
            features=tf.train.Features(
                feature=feature
            )
        )
        examples.append(example.SerializeToString())
    return imported.signatures["predict"](
    examples=tf.constant(examples))

In [468]:
print(predict(prediction_set_n))

{'predictions': <tf.Tensor: id=47650, shape=(6, 1), dtype=float32, numpy=
array([[35.078075],
       [20.180553],
       [24.37819 ],
       [34.91951 ],
       [14.411057],
       [19.973576]], dtype=float32)>}


In [469]:
print("done")

done
